In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import lightgbm as lgbm

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
train.describe()

In [ ]:
plt.figure();

In [ ]:
plt.style.use('ggplot')
columns = list(train)[1:-2]
train[columns].hist(stacked=False, bins=100, figsize=(12,70), layout=(50,4));

if we check at the target variable we can observe the data imbalance . I wanted to try smote to handle this data imbalance

In [ ]:
sm = SMOTE()

Id is not needed for data analysis lets remove it . 
all the data is numerical and not categorical except the target variable 
lets re-lable our data to X , y 

In [ ]:
X = train.drop(['ID_code','target'],axis = 1)
y = train['target']

In [ ]:
Bal_Xtrain,Bal_ytrain = sm.fit_sample(X,y)

In [ ]:
Bal_Xtrain.shape

In [ ]:
X_tb = pd.DataFrame(Bal_Xtrain, columns = X.columns)

In [ ]:
X_tb

In [ ]:
Bal_ytrain.shape

In [ ]:
Y_tb = pd.DataFrame(Bal_ytrain) 

In [ ]:
Y_tb

In [ ]:
Sc = StandardScaler()

In [ ]:
Scaled_Xtrain = Sc.fit_transform(X_tb)

In [ ]:
Scaled_Xtrain = pd.DataFrame(Scaled_Xtrain,columns=X_tb.columns)


In [ ]:
plt.style.use('ggplot')
columns = list(Scaled_Xtrain)[1:-2]
Scaled_Xtrain[columns].hist(stacked=False, bins=100, figsize=(12,70), layout=(50,4));

In [ ]:
Scaled_Xtrain.describe()

logreg = LogisticRegression(class_weight='balanced')
param = {'C':[0.001,0.003,0.005,0.01,0.03,0.05,0.1,0.3,0.5,1,2,3,3,4,5,10,20]}
clf = GridSearchCV(logreg,param,scoring='roc_auc',refit=True,cv=10)
clf.fit(Scaled_Xtrain,Y_tb)
print('Best roc_auc: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_))

skf = StratifiedKFold(n_splits=10)

skf.get_n_splits(Scaled_Xtrain, Y_tb)

In [ ]:
Lr = LogisticRegression(C = 0.001)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(Scaled_Xtrain, Y_tb, test_size=0.33, random_state=42)

In [ ]:
Lr.fit(X_train,y_train)

In [ ]:
predictions = Lr.predict(X_test)

In [ ]:
roc_auc_score(y_test,predictions)

for i in range(len(y_val)):
    if Lassopred[i] >= 0.5:
        Lassopred[i] = 1
    else:
        Lassopred[i] = 0

roc_auc_score(y_val,Lassopred)

In [ ]:
from sklearn.metrics import auc

In [ ]:


fpr, tpr, threshold = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)



plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
test.head()

In [ ]:
X_t = test.drop(['ID_code'], axis = 1)

In [ ]:
scaledX_test = Sc.fit_transform(X_t)

In [ ]:
scaledX_test = pd.DataFrame(scaledX_test,columns= X_t.columns)

In [ ]:
y_t = Lr.predict(scaledX_test)

for i in range(len(y_test)):
    if y_test[i] >= 0.5:
        y_test[i] = 1
    else:
        y_test[i] = 0
     



In [ ]:
submission_Lr = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_t
    })
submission_Lr.to_csv('submission_lr.csv', index=False)